<a href="https://colab.research.google.com/github/DmitryKutsev/DeepHW/blob/master/play_with_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
pip install pandas

In [3]:
!pip install --upgrade gspread gspread_dataframe

In [4]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from collections import Counter

In [5]:
import gspread
from gspread_dataframe import get_as_dataframe
import pandas as pd

In [18]:
data_url = '/reddit-cleanjokes-reddit-cleanjokes.csv'
df = pd.read_csv(data_url, sep=',')

In [10]:
df.head()

,ID,Joke
0,1,What did the bartender say to the jumper cable...
1,2,Don't you hate jokes about German sausage? The...
2,3,Two artists had an art contest... It ended in ...
3,4,Why did the chicken cross the playground? To g...
4,5,What gun do you use to hunt a moose? A moosecut!


In [11]:
class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        sequence_length = 4,
        data_url = '123',
    ):
        self.sequence_length = sequence_length
        self.data_url = data_url

        self.words = self.load_words()
        self.unique_words = self.get_unique_words()

        self.index_to_word = {index: word for index, word in enumerate(self.unique_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.unique_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def load_words(self):
        train_df = pd.read_csv(self.data_url)
        text = train_df['Joke'].str.cat(sep=' ')
        return text.split(' ')

    def get_unique_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - self.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.sequence_length+1]),
        )

In [19]:
data = Dataset(data_url=data_url)

In [20]:
data.__getitem__(1)

(tensor([  8,   0, 248,  20]), tensor([  0, 248,  20,   4]))

In [14]:
class Model(nn.Module):
    def __init__(self, dataset, lstm_size = 128, embedding_dim = 128, num_layers = 3):
        super(Model, self).__init__()
        self.lstm_size = lstm_size
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        n_vocab = len(dataset.unique_words)

        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
            batch_first=True
        )

        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, state):
      embedding = self.embedding(x)
      output, new_state = self.lstm(embedding, state)
      y = self.fc(output)
      return y, new_state

    def init_state(self, batch_size):
      return (torch.zeros(self.num_layers, batch_size, self.lstm_size),
                torch.zeros(self.num_layers, batch_size, self.lstm_size))

In [22]:
def train(model, optimizer, loss_fn_int, train_dataloader, test_dataloader,
          device, epochs = 10, batch_size = 16):

  for epoch in range(epochs):
    state_c, state_h = model.init_state(batch_size)
    state_h = state_h.to(device)
    state_c = state_c.to(device)

    for batch, (x, y) in enumerate(train_dataloader):
      optimizer.zero_grad()

      x = x.to(device)
      y = y.to(device)

      y_pred, (state_h, state_c) = model(x, (state_h, state_c))

      # отсоединяем скрытое состояние от вычислительного графа,
      # чтобы предотвратить его накопление за все эпохи
      state_h = state_h.detach()
      state_c = state_c.detach()

      loss = loss_fn_int(y_pred.transpose(1, 2), y)

      #state_h = state_h.detach()
      #state_c = state_c.detach()

      loss.backward()
      optimizer.step()

      if batch % 100 == 0:
          print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })

In [ ]:
batch_size = 16
epochs = 20

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataset = data
model = Model(dataset = dataset).to(device)
optimizer = optim.Adam(model.parameters())
loss = nn.CrossEntropyLoss()

train_dataloader = DataLoader(dataset, batch_size=batch_size, drop_last=True)
test_dataloader = DataLoader(dataset, batch_size=batch_size, drop_last=True)


train(model, optimizer, loss, train_dataloader, test_dataloader, device, epochs=epochs, batch_size=batch_size)

{'epoch': 0, 'batch': 0, 'loss': 8.823587417602539}
{'epoch': 0, 'batch': 100, 'loss': 7.612061500549316}
{'epoch': 0, 'batch': 200, 'loss': 7.185446739196777}
{'epoch': 0, 'batch': 300, 'loss': 7.621567726135254}
{'epoch': 0, 'batch': 400, 'loss': 7.389347553253174}
{'epoch': 0, 'batch': 500, 'loss': 6.92543888092041}
{'epoch': 0, 'batch': 600, 'loss': 7.110757350921631}
{'epoch': 0, 'batch': 700, 'loss': 7.713561534881592}
{'epoch': 0, 'batch': 800, 'loss': 7.962447643280029}
{'epoch': 0, 'batch': 900, 'loss': 6.424600124359131}
{'epoch': 0, 'batch': 1000, 'loss': 7.436445713043213}
{'epoch': 0, 'batch': 1100, 'loss': 7.006034851074219}
{'epoch': 0, 'batch': 1200, 'loss': 6.221230506896973}
{'epoch': 0, 'batch': 1300, 'loss': 8.683935165405273}
{'epoch': 0, 'batch': 1400, 'loss': 5.996964454650879}
{'epoch': 1, 'batch': 0, 'loss': 6.747366428375244}
{'epoch': 1, 'batch': 100, 'loss': 6.368834495544434}
{'epoch': 1, 'batch': 200, 'loss': 6.056544303894043}
{'epoch': 1, 'batch': 300, '

In [ ]:
def predict(dataset, model, text, next_words=10):
    #your code here